# Nested NER with spaCy Spancat

In [1]:
import spacy

## Data Preparation

In [16]:
#config = 'ggponc2_fine_long_bigbio_kb'
config = 'ggponc2_fine_short_bigbio_kb'

In [17]:
from datasets import load_dataset
ggponc_raw = load_dataset('bigbio/ggponc2', data_dir='data/v2.0_2022_03_24', name=config)

Generating train split: 7135 examples [00:11, 608.01 examples/s] 
Generating test split: 1529 examples [00:08, 174.64 examples/s]
Generating validation split: 1529 examples [00:08, 173.77 examples/s]


In [18]:
from ner_util import *

ggponc_ds = bigbio_split_passages(ggponc_raw)
ggponc_ds

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1529/1529 [00:01<00:00, 1036.73 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 59515
    })
    test: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 13714
    })
    validation: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 12770
    })
})

In [19]:
from xmen.data import from_spacy

nlp = spacy.blank('de')

#### Upsampling 

We oversample (3x) documents with rare classes (Nutrient / Body Substance and External Substance), which are otherwise tend to be ignored during training

In [20]:
output = Path('data') / config
output.mkdir(exist_ok=True)

In [21]:
bigbio_to_spacy_docbin(output / 'ggponc_spacy', nlp, ggponc_ds, 'entities', is_sentencized=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████▉| 12769/12770 [00:02<00:00, 4719.08it/s]


In [22]:
import datasets

substance_examples = ggponc_ds['train'].filter(lambda d: len([e for e in d['entities'] if 'Substance' in e['type']]) > 0)
substance_examples

ggponc_ds_upsampled = ggponc_ds.copy()
ggponc_ds_upsampled['train'] = datasets.concatenate_datasets([ggponc_ds['train'], substance_examples, substance_examples])
ggponc_ds_upsampled

Filter: 100%|███████████████████████████████████████████████████████████████████████████████| 59515/59515 [00:04<00:00, 14179.10 examples/s]


{'train': Dataset({
     features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
     num_rows: 63619
 }),
 'test': Dataset({
     features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
     num_rows: 13714
 }),
 'validation': Dataset({
     features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
     num_rows: 12770
 })}

In [23]:
bigbio_to_spacy_docbin(output / 'ggponc_spacy_up', nlp, ggponc_ds_upsampled, 'entities', is_sentencized=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████▉| 12769/12770 [00:02<00:00, 5726.28it/s]


## Train spaCy Model

see: [./spacy_ner/run_training.sh](./spacy_ner/run_training.sh)

**Optional**: build spaCy package, e.g.:

`python -m spacy package spacy_ner/training/2023-12-27_18-02-08-cuda-4/model-best/ models --code spacy_ner/chunk_and_ngram_suggester.py --name ggponc_medbertde --build wheel --create-meta`

## NER Predictions

In [10]:
ner_model = spacy.load('de_ggponc_medbertde')

## or
# run = 'oga0cw1e'
# ner_model = spacy.load(f'spacy_ner/training/{run}/model-best')

In [11]:
from spacy import displacy

def show(s):
    d = ner_model(s)
    displacy.render(d, style="span", options={'spans_key' : 'entities'})

In [12]:
show("Versagen einer Behandlung mit Oxaliplatin und Irinotecan")

In [13]:
show(""""Cetuximab ist ein monoklonaler Antikörper, der gegen den epidermalen Wachstumsfaktorrezeptor (EGFR) gerichtet ist und 
dient zur Therapie des fortgeschrittenen kolorektalen Karzinoms zusammen mit Irinotecan oder in Kombination mit FOLFOX bzw. allein nach Versagen einer Behandlung mit Oxaliplatin und Irinotecan.""")

In [18]:
show("""Antibiose fortsetzen (s. o.), Abstrich erfragen, ggf. Umstellung der Antibiose. Thromboseprophylaxe bis zur sicheren Mobilität.""")

In [19]:
show("""
Die übrigen Infizierten sind beschwerdefrei und zeigen keine Symptome; sie sind asymptomatisch erkrankt, können aber dennoch das Virus weiterverbreiten.[11][12] Bei rund 81 % der registrierten Erkrankungen ist ein leichter Verlauf mit Fieber oder einer leichten Lungenentzündung, trockenem Husten und Müdigkeit zu beobachten. Weniger häufig sind eine verstopfte Nase, Kopfschmerzen, Halsschmerzen, Gliederschmerzen, Bindehautentzündungen, Durchfall, Erbrechen, Geschmacks- und Geruchsverlust, Hautausschlag oder Verfärbung von Fingern oder Zehen. Bei etwa 14 % der Krankheitsfälle ist der Verlauf schwerer, und in etwa 5 % so schwer, dass eine Beatmung der Patienten auf einer Intensivstation erfolgen muss.""")

In [20]:
show("""
Anamnese. Der 67-jährige Patient ist bekannt in Ihrer allgemeininternistischen Praxis. Letzte Vorstellung vor 2 Monaten: Erstdiagnose Magenkarzinom, Beginn neoadjuvante Chemotherapie (FLOT-Protokoll mit 5‑Fluorouracil, Folinsäure, Oxaliplatin und Docetaxel) in potenziell kurativem Setting im lokalen Klinikum
Heutige Vorstellung: seit etwa 10 Tagen zunehmende Dyspnoe (erst unter Belastung, mittlerweile auch in Ruhe, Orthopnoe nachts). Gelegentlich etwas trockener Husten, zweimalig Temperatur von 37,7 °C in den letzten 10 Tagen. In 3 Tagen steht der nächste Chemotherapiezyklus an. Der Patient legt einen Arztbrief vor. Inhalt: stationäre Behandlung vor 3 Wochen aufgrund einer Pneumonie (linksseitig)

Vorerkrankungen. Linksherzinsuffizienz („heart failure with preserved ejection fraction“ [HFpEF], linksventrikuläre Ejektionsfraktion 50 %), chronisch-obstruktive Lungenerkrankung im Stadium I nach Global Initiative for Chronic Obstructive Lung Disease (GOLD), Risikoklasse A, florider Nikotinabusus (kumulativ 20 Packungsjahre), Magenkarzinom Stadium IIA
Körperliche Untersuchung. Auskultation: beidseits vesikuläres Atemgeräusch mit gering verlängertem Exspirium und basaler Dämpfung links, keine Rasselgeräusche. Perkussion: sonorer Klopfschall bis auf links basal – hier hyposonor, Lungengrenze links nicht atemverschieblich. Herztöne rhythmisch, rein und normofrequent. Knöchelödeme
""")

## Evaluate NER Model

In [21]:
from spacy.tokens import DocBin
from itertools import islice
from xmen.data import from_spacy

spacy_test_db = DocBin(store_user_data=True).from_disk('data/ggponc2_fine_long_bigbio_kb/ggponc_spacy/test.spacy')
spacy_test = list(spacy_test_db.get_docs(ner_model.vocab))

In [22]:
spacy_test_ds = from_spacy(spacy_test, 'entities')

In [23]:
from tqdm.auto import tqdm
test_data = [d.text for d in spacy_test]
pred = list(tqdm(ner_model.pipe(test_data,  batch_size=128), total=len(test_data)))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 13714/13714 [06:41<00:00, 34.14it/s]


In [24]:
pred_ds = from_spacy(pred, 'entities')

In [25]:
from xmen.evaluation import evaluate, error_analysis
evaluate(spacy_test_ds, pred_ds, ner_only=True, metrics=['strict', 'partial'])

{'strict': {'precision': 0.7304198171763909,
  'recall': 0.7528717247797133,
  'fscore': 0.7414758488350446,
  'ptp': 25889,
  'fp': 9555,
  'rtp': 25889,
  'fn': 8498,
  'n_docs_system': 13714,
  'n_annos_system': 35507,
  'n_docs_gold': 13714,
  'n_annos_gold': 34414},
 'partial': {'precision': 0.8292610477773855,
  'recall': 0.8283757480739823,
  'fscore': 0.8288181615181769,
  'ptp': 29444.57202343163,
  'fp': 6062.427976568371,
  'rtp': 28507.722994218027,
  'fn': 5906.277005781973,
  'n_docs_system': 13714,
  'n_annos_system': 35507,
  'n_docs_gold': 13714,
  'n_annos_gold': 34414}}

# Error Analysis

In [26]:
ea = error_analysis(spacy_test_ds, pred_ds, tasks=['ner'])

In [27]:
ea.ner_match_type.value_counts()

ner_match_type
tp     25889
be      5137
fn      2355
fp      2010
lbe     1774
le      1643
Name: count, dtype: int64